In [1]:
#imports needed
import os
import torch
from torch import nn
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error 

In [2]:
#load data
boston_dataset = load_boston()
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston['MEDV'] = boston_dataset.target
X = boston.loc[:, boston.columns != 'MEDV']
Y = boston['MEDV']


d:\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.
    :fun

In [3]:
#split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state=5)
X_train = torch.tensor(X_train.values).float()
X_test = torch.tensor(X_test.values).float()
Y_train = torch.tensor(Y_train.values).float()
Y_test = torch.tensor(Y_test.values).float()


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(13,16,True)
        self.fc2 = nn.Linear(16,32,True)
        self.fc3 = nn.Linear(32,1,True)
        

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
net = NeuralNetwork()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.0000008)

for epoch in range(150): 
    for data in range(len(X_train)):
        optimizer.zero_grad()
        outputs = net(X_train[data])
        loss = criterion(outputs.squeeze(), Y_train[data])
        loss.backward()
        optimizer.step()

In [6]:
# model evaluation for train set
y_train_predict = net(X_train)  
y_train_predict = y_train_predict.detach().numpy()
rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))
print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print("\n")

The model performance for training set
--------------------------------------
RMSE is 8.479241371154785




In [7]:

# model evaluation for testing set
y_test_predict = net(X_test)
y_test_predict = y_test_predict.detach().numpy()
rmse = (np.sqrt(mean_squared_error(Y_test.squeeze(), y_test_predict)))

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))

The model performance for testing set
--------------------------------------
RMSE is 9.35856819152832
